In [1]:
%jsroot on
TFile *ipf = new TFile("root/cchist.root");
TCanvas *c1 = new TCanvas;
c1->SetLogy(0);

In [2]:
h2edt->Draw("colz");
c1->Draw();

In [3]:
Double_t lambda[4];
Double_t Pb[3];
Double_t Pn[2];

Pb[0] = 1;//母核的beta衰变分支比
Pn[0] = 1-Pb[0];//母核的beta-n衰变分支比
//Pb[1] = Pb[0];//beta子核的beta衰变分支比
//Pn[1] = 1-Pb[1];//beta子核的beta-n衰变分支比
//Pb[2] = Pb[0];//beta-n子核的beta衰变分支比

lambda[0]=log(2)/840;//beta子核的衰变常数
lambda[1]=log(2)/1090;//beta-n子核的衰变常数
//lambda[2]=log(2)/840;//beta孙子核的衰变常数
//lambda[3]=log(2)/1090;//beta-n子核的beta子核的衰变常数

In [4]:
TString s1 = "[0]+[1]*TMath::Exp(-x*log(2)/[2])";
TString s2, s5, s3, s61, s62;
s2.Form("+ [1] * %f * %f * (TMath::Exp(-log(2)/[2] * x) / (%f - log(2)/[2]) + TMath::Exp(-%f * x) / (log(2)/[2] - %f))",
        Pb[0], lambda[0], lambda[0], lambda[0], lambda[0]);
s5.Form("+ [1] * %f * %f * (TMath::Exp(-log(2)/[2] * x) / (%f - log(2)/[2]) + TMath::Exp(-%f * x) / (log(2)/[2] - %f))",
        Pn[0], lambda[1], lambda[1], lambda[1], lambda[1]);
/*
s3.Form("+ [1] * %f * %f * %f * %f * (TMath::Exp(-log(2)/[2] * x) / ((%f - log(2)/[2])*(%f - log(2)/[2])) + TMath::Exp(-%f * x) / ((log(2)/[2] - %f)*(%f - %f))+ TMath::Exp(-%f * x) / ((log(2)/[2] - %f)*(%f - %f)))",
        Pb[0], Pb[1], lambda[0], lambda[2], lambda[0], lambda[2], lambda[0], lambda[0], lambda[2], lambda[0], lambda[2], lambda[2], lambda[0], lambda[2] );
s61.Form("+ [1] * %f * %f * %f * %f * (TMath::Exp(-log(2)/[2] * x) / ((%f - log(2)/[2])*(%f - log(2)/[2])) + TMath::Exp(-%f * x) / ((log(2)/[2] - %f)*(%f - %f))+ TMath::Exp(-%f * x) / ((log(2)/[2] - %f)*(%f - %f)))",
         Pb[0], Pn[1], lambda[0], lambda[3], lambda[0], lambda[3], lambda[0], lambda[0], lambda[3], lambda[0], lambda[3], lambda[3], lambda[0], lambda[3]);
s62.Form("+ [1] * %f * %f * %f * %f * (TMath::Exp(-log(2)/[2] * x) / ((%f - log(2)/[2])*(%f - log(2)/[2])) + TMath::Exp(-%f * x) / ((log(2)/[2] - %f)*(%f - %f))+ TMath::Exp(-%f * x) / ((log(2)/[2] - %f)*(%f - %f)))",
         Pn[0], Pb[2], lambda[1], lambda[3], lambda[1], lambda[3], lambda[1], lambda[1], lambda[3], lambda[1], lambda[3], lambda[3], lambda[1], lambda[3]);
*/

In [5]:
TF1 *f1 = new TF1("f1",s1+s2+s5,0,4500);  // parameter [2] is T1/2
f1->SetNpx(4000);

In [6]:
Int_t e,e1,e2;  // gate (e1,e2), name e
vector<int> b;  // background bin

In [7]:
void gate() // nbin: decaytime's bin; xmax: fit range (0,xmax)
{
    h2edt->ProjectionY(Form("h%d",e),e1+1,e2+1);
    TH1F *hdt = (TH1F*)gROOT->FindObject(Form("h%d",e));
    //hdt->Sumw2();
    TH1F *hbg = new TH1F(Form("hbg%d",e),"hbg",h2edt->GetYaxis()->GetNbins(),0,h2edt->GetYaxis()->GetXmax());
    for (unsigned int i=0; i<b.size(); i++){
        h2edt->ProjectionY(Form("b%d",b[i]),b[i]+1,b[i]+1);
        TH1F *hbtemp = (TH1F*)gROOT->FindObject(Form("b%d",b[i]));
        hbg->Add(hbg,hbtemp,1,1);
    }
    double weight = (double)(e2-e1+1)/b.size();
    hdt->Add(hdt,hbg,1,-weight);
    
    f1->SetParameter(0,1000);
    f1->SetParameter(1,100);
    f1->SetParameter(2,50);
    hdt->Fit("f1","R");
    hdt->Draw();
    c1->Draw();
    delete hbg;
}

In [8]:
h2edt->ProjectionX("tpjx");
tpjx->Draw();
c1->Draw();

In [9]:
//%jsroot off
c1->SetLogy(0);

## 128Cd  T1/2 = 280 ms

In [10]:
e = 645;
e1 = 643;
e2 = 646;
b = {628,615,663,671};
gate();

 FCN=67.574 FROM MIGRAD    STATUS=CONVERGED     140 CALLS         141 TOTAL
                     EDM=1.13009e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           2.05989e+03   1.11001e+01   2.82544e-02  -1.32135e-05
   2  p1           2.55268e+03   5.52620e+01   1.19197e-01  -8.04010e-06
   3  p2           2.63503e+02   7.85561e+00   1.37837e-02  -2.40751e-04


In [11]:
e = 784;
e1 = 782;
e2 = 785;
b = {749,760,800,827};
gate();

 FCN=53.9647 FROM MIGRAD    STATUS=CONVERGED      92 CALLS          93 TOTAL
                     EDM=2.43189e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.23479e+03   8.48384e+00   1.95971e-02   3.97300e-05
   2  p1           1.55951e+03   4.23613e+01   8.33053e-02   4.59640e-06
   3  p2           2.61337e+02   9.60720e+00   1.55558e-02   3.13212e-05
